<a href="https://colab.research.google.com/github/rajaswa/Summer-Internship-IIT-Bombay-2019/blob/master/lda_mallet_reddit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Downloading Data

In [1]:
!wget --header="Host: codeload.github.com" --header="User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.131 Safari/537.36" --header="Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3" --header="Accept-Language: en-US,en;q=0.9,lb;q=0.8" --header="Referer: https://github.com/linanqiu/reddit-dataset" --header="Cookie: _octo=GH1.1.1541950956.1513609300; _ga=GA1.2.2088113052.1513609300; logged_in=yes; dotcom_user=rajaswa; _gat=1" --header="Connection: keep-alive" "https://codeload.github.com/linanqiu/reddit-dataset/zip/master" -O "CurlWget1684" -c

--2019-05-17 18:22:36--  https://codeload.github.com/linanqiu/reddit-dataset/zip/master
Resolving codeload.github.com (codeload.github.com)... 192.30.255.120
Connecting to codeload.github.com (codeload.github.com)|192.30.255.120|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘CurlWget1684’

CurlWget1684            [                <=> ] 242.66M  6.37MB/s    in 41s     

2019-05-17 18:23:17 (5.93 MB/s) - ‘CurlWget1684’ saved [254445121]



In [2]:
!unzip CurlWget1684

Archive:  CurlWget1684
d2f7dc81c1bae4238805ed5b20fdaaee45f71cde
   creating: reddit-dataset-master/
 extracting: reddit-dataset-master/.gitignore  
  inflating: reddit-dataset-master/README.md  
  inflating: reddit-dataset-master/entertainment_anime.csv  
  inflating: reddit-dataset-master/entertainment_comicbooks.csv  
  inflating: reddit-dataset-master/entertainment_harrypotter.csv  
  inflating: reddit-dataset-master/entertainment_movies.csv  
  inflating: reddit-dataset-master/entertainment_music.csv  
  inflating: reddit-dataset-master/entertainment_starwars.csv  
  inflating: reddit-dataset-master/gaming_dota2.csv  
  inflating: reddit-dataset-master/gaming_gaming.csv  
  inflating: reddit-dataset-master/gaming_leagueoflegends.csv  
  inflating: reddit-dataset-master/gaming_minecraft.csv  
  inflating: reddit-dataset-master/gaming_pokemon.csv  
  inflating: reddit-dataset-master/gaming_skyrim.csv  
  inflating: reddit-dataset-master/gaming_starcraft.csv  
  inflating: reddit-data

In [3]:
!ls -R

.:
CurlWget1684  reddit-dataset-master  sample_data

./reddit-dataset-master:
entertainment_anime.csv		learning_youshouldknow.csv
entertainment_comicbooks.csv	lifestyle_drunk.csv
entertainment_harrypotter.csv	lifestyle_food.csv
entertainment_movies.csv	lifestyle_frugal.csv
entertainment_music.csv		lifestyle_guns.csv
entertainment_starwars.csv	lifestyle_lifehacks.csv
gaming_dota2.csv		lifestyle_motorcycles.csv
gaming_gaming.csv		lifestyle_progresspics.csv
gaming_leagueoflegends.csv	lifestyle_sex.csv
gaming_minecraft.csv		news_conservative.csv
gaming_pokemon.csv		news_conspiracy.csv
gaming_skyrim.csv		news_libertarian.csv
gaming_starcraft.csv		news_news.csv
gaming_tf2.csv			news_offbeat.csv
headers.txt			news_politics.csv
humor_adviceanimals.csv		news_truereddit.csv
humor_circlejerk.csv		news_worldnews.csv
humor_facepalm.csv		README.md
humor_funny.csv			television_breakingbad.csv
humor_imgoingtohellforthis.csv	television_community.csv
humor_jokes.csv			television_doctorwho.csv
learning_a

# Installing Dependencies

In [4]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 4.3MB/s 
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [5]:
import nltk
nltk.download('stopwords')

!python3 -m spacy download en

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.

    Linking successful
    /usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/en

    You can now load the model via spacy.load('en')



# Imports

In [0]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [0]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [8]:
#loading as pandas dataframe

df = pd.read_csv('./reddit-dataset-master/television_gameofthrones.csv')
df = df.dropna()
print(len(df))
df[:20]

41021


,Unnamed: 0,0,1,2,3,4,5,6,7,8,9,10
0,0,i have a theory that there are too many theore...,d02pnrs,gameofthrones,television,1.455676e+09,OldTrick16,2.0,0.0,1,727,0.0
1,1,they ve all killed each other without acknowle...,d02ob47,gameofthrones,television,1.455674e+09,ColdBeerz,1.0,0.0,348,42747,0.0
2,2,this is a reminder that theories still need to...,d023jp8,gameofthrones,television,1.455644e+09,AutoModerator,1.0,0.0,1476,278459,1.0
3,3,stark does nt matter they have plot armor they...,d024tpe,gameofthrones,television,1.455646e+09,mking1999,34.0,0.0,1,7452,0.0
4,4,i could and would argue that the seven great h...,d02m8j2,gameofthrones,television,1.455670e+09,lowlzmclovin,1.0,0.0,6,938,0.0
5,5,daenerys does own lands meereen i think,d02cjg7,gameofthrones,television,1.455656e+09,GeelongJr,2.0,0.0,148,1046,0.0
6,6,stannis could still be alive,d02bb2e,gameofthrones,television,1.455655e+09,JSouth25,2.0,0.0,1,248,0.0
7,7,there s also the lannisport lannisters if the ...,d028vse,gameofthrones,television,1.455651e+09,Shepherdsfavestore,4.0,0.0,90,16303,0.0
8,8,this post has a no spoilers title warning w...,d00y9k1,gameofthrones,television,1.455564e+09,AutoModerator,3.0,0.0,1476,278459,1.0
9,9,i must be the only person on reddit who has ne...,d01i0k2,gameofthrones,television,1.455594e+09,TheBaronOfTheNorth,1.0,0.0,1,17791,0.0


In [9]:
#text preprocessing

# Convert to list
data = df['0'].values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['i have a theory that there are too many theoretical theories going around '
 'how s that for a theory ']


In [10]:
#tokenizing text

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['have', 'theory', 'that', 'there', 'are', 'too', 'many', 'theoretical', 'theories', 'going', 'around', 'how', 'that', 'for', 'theory']]


In [11]:
#building bigram & trigram models for frequently occuring n-grams

# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['have', 'theory', 'that', 'there', 'are', 'too', 'many', 'theoretical', 'theories', 'going', 'around', 'how', 'that', 'for', 'theory']


In [0]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['theory', 'many', 'theoretical', 'theory', 'go', 'theory']]


In [19]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 3)]]


# LDA Mallet Model

In [15]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip

--2019-05-17 18:30:21--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  12.4MB/s    in 1.2s    

2019-05-17 18:30:23 (12.4 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]



In [0]:
!unzip mallet-2.0.8.zip 

In [17]:
!ls -R

.:
CurlWget1684  mallet-2.0.8.zip	     sample_data
mallet-2.0.8  reddit-dataset-master

./mallet-2.0.8:
bin	   class  lib	   Makefile  README.md	  src	     test
build.xml  dist   LICENSE  pom.xml   sample-data  stoplists

./mallet-2.0.8/bin:
classifier2info  mallet.bat	     text2classify     vectors2topics
csv2classify	 mallethon	     text2vectors      vectors2vectors
csv2vectors	 prepend-license.sh  vectors2classify
mallet		 svmlight2vectors    vectors2info

./mallet-2.0.8/class:
cc

./mallet-2.0.8/class/cc:
mallet

./mallet-2.0.8/class/cc/mallet:
classify  examples  fst   optimize  regression	topics	util
cluster   extract   grmm  pipe	    share	types

./mallet-2.0.8/class/cc/mallet/classify:
 AdaBoost.class
 AdaBoostM2.class
 AdaBoostM2Trainer.class
 AdaBoostTrainer.class
 BaggingClassifier.class
 BaggingTrainer.class
 BalancedWinnow.class
 BalancedWinnowTrainer.class
 Boostable.class
 C45.class
'C45$Node.class'
 C45Trainer.class
 Classification.class
 ClassifierAccuracyEvaluator.cla

In [0]:
mallet_path = './mallet-2.0.8/bin/mallet' # update this path
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=20, id2word=id2word)

In [21]:
# Show Topics
pprint(ldamallet.show_topics(formatted=False))

# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)

[(19,
  [('wall', 0.03216772630941568),
   ('sam', 0.030805328489252194),
   ('watch', 0.030464729034211324),
   ('leave', 0.029556463820769),
   ('wait', 0.028231910384498942),
   ('episode', 0.025393581592491676),
   ('black', 0.02452316076294278),
   ('castle', 0.02285800787163185),
   ('nice', 0.020360278534665457),
   ('remember', 0.020284589766878594)]),
 (13,
  [('nt', 0.07409460458240946),
   ('baby', 0.030303030303030304),
   ('people', 0.028307464892830748),
   ('kill', 0.026940133037694013),
   ('king', 0.022283813747228383),
   ('person', 0.02187730968218773),
   ('post', 0.01770140428677014),
   ('time', 0.016444937176644494),
   ('boy', 0.015373244641537324),
   ('cersei', 0.014597191426459719)]),
 (18,
  [('player', 0.02329909698766113),
   ('character', 0.02133452815881988),
   ('show', 0.020128213965671746),
   ('time', 0.01723305990211622),
   ('game', 0.01695733094368236),
   ('lady', 0.015509753911904598),
   ('make', 0.014613634796994554),
   ('find', 0.01440683807